In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from matplotlib.pyplot import figure

from pylab import rcParams

LABELS = ["Normal", "Fraude"]

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

In [2]:
df = pd.read_csv("creditcard.csv")

X = np.array(df.iloc[:,1:-1])
y = np.array(df['Class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

## SVM

In [5]:
from sklearn import svm

Datos balanceados

In [ ]:
clf = svm.SVC(kernel='linear',class_weight={0:1,1:1} )
clf.fit(X_res, y_res)
yhat5 = clf.predict(X_test)
print(classification_report(y_test, yhat5))

In [ ]:
print('roc_auc_score for KNN balanceado: ', roc_auc_score(y_test, yhat5))

## XG Boost

In [3]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

xgbmodel = XGBRegressor()
xgbmodel.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'enable_categorical': False,
 'gamma': None,
 'gpu_id': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [4]:
params={ 'base_score': [0.5], # prediccion inicial
     'booster': ['gbtree'],# (gbtree, gblinear, dart)
     'colsample_bylevel': [1],
     'colsample_bytree': [0.8],
     'learning_rate': [0.05,0.1], 
     'max_depth': [3], 
     'min_child_weight': [1], # minimo numero samples por hoja
     'missing': [np.nan], # si queremos reemplazar los missings por un numero
     'n_estimators': [100,150,200,250,300,350,400], # numero de arboles
     'n_jobs': [-1], # trabajos en paralelo
     'objective': ['reg:logistic'],#  Output. Tipo de función que estamos estimando
     'random_state': [0], # seed para generar los folds
     'alpha': [0.01], # L1 regularitacion
     'lambda': [0.01], # L2 regularitacion
     'scale_pos_weight': [1],
     'subsample': [0.9]} # ratio de muestras por cada arbol 


scoring = ['recall']
grid_solver = GridSearchCV(estimator = xgbmodel, # model to train
                   param_grid = params,
                   scoring = scoring,
                   cv = 5,
                   n_jobs=-1,
                   refit = 'recall',
                   verbose = 2)

model_result_xgboost = grid_solver.fit(X_res, y_res)

model_result_xgboost.best_params_

Fitting 5 folds for each of 14 candidates, totalling 70 fits


/Users/juanjovalero/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/juanjovalero/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/juanjovalero/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "/Users/juanjovalero/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "/Users/juanjovalero/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/Users/juanjovalero/opt/anaconda3/lib/python3.9/site-packa

KeyboardInterrupt: 

In [ ]:
yhat_xgboost=model_result_xgboost.predict(X_test)
print(classification_report(y_test, yhat_xgboost))

In [ ]:
results_cv=pd.DataFrame(model_result_xgboost.cv_results_)
sns.relplot(x='param_n_estimators',
            y='mean_test_neg_mean_squared_error',
            style='param_learning_rate',
            palette=['r','g'],
            kind='line',data=results_cv)